Level 1 and Level 3

In [199]:
import requests
#first I just built the API connection to Mistral, 
#so that I know that it works
apiKey = "fbEJEobe12xbsYUEUtXAo1VZ9LBGqFjz"
#this was renamed to askMistralNode with state as input l8r

import json
import os

MEMORY_PATH = "memory.json"
# so very basic memory system, just a json file
#that stores as many messages as you want
def loadMemory():
    if not os.path.exists(MEMORY_PATH):
        return {"messages": [], "counter": 0, "persona": ""}
    with open(MEMORY_PATH, "r") as f:
        return json.load(f)

def saveMemory(data):
    with open(MEMORY_PATH, "w") as f:
        json.dump(data, f, indent=2)
        
        
def askMistralNode(state):
    memory = loadMemory()
    msg = state["input"]
    
    memory["messages"].append(msg)
    memory["messages"] = memory["messages"][-12:]  
    memory["counter"] += 1
    
    context = "\n".join(memory["messages"][-3:])
    
    if memory["counter"] % 6 == 0:
        chPrompt = (
            "These are recent user prompts. Summarize the user's interests or personality "
            "as an assistant might find useful to tailor future replies. Dive into the summary directly.:\n" +
            "\n".join(memory["messages"][-6:-1])
        )
        res = requests.post(
            "https://api.mistral.ai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {apiKey}",
                "Content-Type": "application/json"
            },
            json={
                "model": "mistral-medium",
                "messages": [
                    {"role": "system", "content": "You summarize users from their chat logs."},
                    {"role": "user", "content": chPrompt}
                ]
            }
        )

        persona = res.json()["choices"][0]["message"]["content"]
        memory["persona"] = persona
    
    saveMemory(memory)

    totPrompt = (
        f"[You know the following about the user: {memory.get('persona', '').strip()}]\n"
        f"[Recent context: {context}]\n"
        f"{msg}"
    )

    res = requests.post(
        "https://api.mistral.ai/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {apiKey}",
            "Content-Type": "application/json"
        },
        json={
            "model": "mistral-medium",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant that analyzes user requests. If the user is asking for a weather forecast, reply with 'weather' on the first line, followed by the precise location (city/district/place, prefecture/province/county). If the user is asking for a mathematical calculation, reply with 'calc' on the first line. If the user is asking for fashion trends, reply with 'fashion' on the first line, followed by the precise location (city/district/place, prefecture/province/sector). For all other requests, reply with 'llm' on the first line and then respond normally."},
                {"role": "user", "content": totPrompt}
            ]
        }
    )
    
    reply = res.json()["choices"][0]["message"]["content"]
    lines = reply.strip().split("\n")
    task = lines[0].strip().lower()
    loc = ""
    for i in range(1, len(lines)):
        loc += lines[i].strip() + ", "
    loc = loc[:-2] if (task == "weather" or task == "fashion") and len(lines) > 1 else None
    
    return {"output": reply, "task": task, "location": loc, "input": msg}

from langgraph.graph import StateGraph, START, END
import re

def decide(state):
    return state.get("task", "llm")
#so the router  returns llm by default if no agentic pathway called

def doMathNode(state):
    "do maffs"
    input = state["input"]
    maffs = re.split(r'[a-zA-Z!?,":;_]+', input)
    maffs = [m.strip() for m in maffs if m.strip()]
    soln = ""
    
    if maffs:
        i = 1
        for s in maffs:
            if re.match(r'^[\d\.\+\-\*/\(\)\s]+$', s):
                try:
                    result = eval(s)
                    soln += (f"{i}th expression is {result}\n")
                    i += 1
                except Exception as e:
                    soln += (f"Math Error in '{s}': {str(e)}\n")
            else:
                continue
        
        if i == 1:
            return {"output": "no math found", "input": input}
        return {"output": "math processing completed.\n" + soln, "input": input}
    else:
        return {"output": "no math found", "input": input}

graph = StateGraph(dict)

graph.add_node("llm", askMistralNode)
graph.add_edge(START, "llm")
#so we start at askMistralNode, and then we go to the decision node
#but router node is kind of a dummy == node, just because in askMistralNode
#we have the ACTUAL functionality of calling the AI (n so we dont call AI twice)
#more of graph explained next cell
graph.add_edge("llm", "router")
graph.add_node("calc", doMathNode)
graph.add_edge("calc", END)

import networkx as nx
import matplotlib.pyplot as plt

Level 2

In [200]:
from urllib.parse import quote
#hey i just learned urllib.parse.quote is a thing
def getWeather(state):
    loc = state.get("location")
    if not loc:
        return {"output": "No location found.", "input": state["input"]}
    #so one of the earlier things i tried was opencage ai (because locations can be like LAX or London, ohio
    # or the Bronx or whatever (so we DO need a geocoder for better fidelity)
    # but opencage was not working for ONLY Asian/African/East Europe so locationiq it is)
    print(loc)
    geo = requests.get(f"https://eu1.locationiq.com/v1/search?key=pk.22b59a75499c47d8801ed033a5d1dab0&q={quote(loc)}&format=json")
    # if geo.status_code != 200:
    #     return {"output": "Geo lookup failed", "input": state["input"]}
    
    try:
        res = geo.json()[0]
        lat = float(res["lat"])
        lng = float(res["lon"])
    except Exception as e:
        return {"output": f"Geocoding error: {e}", "input": state["input"]}
    
    url = f"http://api.weatherapi.com/v1/forecast.json?key=7bdb0a1fd8ba433c98061921252105&q={lat},{lng}&days=10"
    #Im using weatherapis forecast API which is very neat in its data and allowed me to make sexy graphs
    try:
        weather = requests.get(url)
        if weather.status_code == 200:
            weatherData = weather.json()
            summary = processWeatherData(weatherData)
            return {"output": summary, "weatherData": weatherData, "input": state["input"]}
        return {"output": f"Weather error {weather.status_code}", "input": state["input"]}
    except Exception as e:
        return {"output": f"Exception: {e}", "input": state["input"]}


def dmsToDecimal(dms):
    d, m, s = map(float, re.findall(r'(\d+\.?\d*)', dms))
    return round(d + m / 60 + s / 3600, 6)

graph.add_node("weather", getWeather)
graph.add_node("router", lambda x: x)

graph.add_edge("weather", END)

In [201]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def getFashion(state):
    loc = state.get("location")
    if not loc:
        return {"output": "No location found.", "input": state["input"]}

    q = f"current fashion trends {loc.split(',')[0]}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    try:
        url = f"https://www.google.com/search?q={quote(q)}"
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")
        
        srs = [
            "div[data-attrid='wa:/description']",
            "div[data-attrid='kc:/fashion/fashion:short description']",
            "div[data-sncf]"
        ]
        overview = "No overview found."
        for s in srs:
            divs = soup.select(s)
            if divs:
                overview = divs[0].get_text().strip()
                break
    except Exception as e:
        overview = f"Overview error: {e}"

    try:
        imgUrl = f"https://www.google.com/search?q={quote(q)}&tbm=isch&tbs=qdr:w"
        r = requests.get(imgUrl, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")
        imgLinks = [img.get("src") for img in soup.select("img") 
                   if img.get("src") and "http" in img.get("src")][:10]
    except Exception as e:
        imgLinks = [f"Image error: {e}"]

    return {"output": overview, "images": imgLinks, "input": state["input"]}


graph.add_node("fashion", getFashion)

graph.add_conditional_edges("router", decide, {
    "weather": "weather", 
    "calc": "calc", 
    "fashion": "fashion",
    "llm": END
})
#this is the ^^^ heart of the graph. delete this for fun stuff.
graph.add_edge("fashion", END)
weatherMind = graph.compile()

# weatherMind.get_graph().draw()


Level 2

In [202]:
import json
import matplotlib.pyplot as plt


def processWeatherData(weatherData):
    location = weatherData.get("location", {})
    name = location.get("name", "Unknown")
    region = location.get("region", "Unknown")
    country = location.get("country", "Unknown")
    localtime = location.get("localtime", "Unknown")

    summary = f"\n=== Weather Forecast from Station at {name}, {region}, {country} ===\n"
    summary += f"Local Time: {localtime}\n"
    
    forecastDays = weatherData.get("forecast", {}).get("forecastday", [])
    for day in forecastDays:
        date = day["date"]
        maxTemp = day["day"]["maxtemp_c"]
        minTemp = day["day"]["mintemp_c"]
        avgTemp = day["day"]["avgtemp_c"]
        avgTempF = day["day"]["avgtemp_f"]
        maxWind = day["day"]["maxwind_kph"]
        maxWindM = day["day"]["maxwind_mph"]
        totalPrecip = day["day"]["totalprecip_mm"]
        totalPrecipI = day["day"]["totalprecip_in"]
        snow = day["day"].get("totalsnow_cm", 0)
        snowI = day["day"].get("totalsnow_in", 0)
        condition = day["day"]["condition"]["text"]
        chanceRain = day["day"].get("daily_chance_of_rain", "N/A")
        chanceSnow = day["day"].get("daily_chance_of_snow", "N/A")

        summary += f"\n### {date} - {condition}\n" + \
        f"Temp: {avgTemp}°C (avg) [{avgTempF}°F], {maxTemp}°C/{minTemp}°C (high/low)\n" + \
        f"Wind: {maxWind} kph [{maxWindM} mph]\n" + \
        f"Precipitation: {totalPrecip} mm [{totalPrecipI} in]\n" + \
        f"Snow: {snow} cm [{snowI} in]\n" + \
        f"Chance of Rain: {chanceRain}%\n" + \
        f"Chance of Snow: {chanceSnow}%\n"
    
    return summary

def plotWeatherGraphs(weatherData):
    hours = []
    hour_labels = []
    tempC = []
    feelslikeC = []
    windKph = []
    gustKph = []
    precipMm = []
    snowCm = []
    humidity = []
    cloud = []
    windchillC = []
    heatindexC = []
    dewpointC = []
    rainChance = []
    snowChance = []
    visibilityKm = []
    uvIndex = []

    dayends = [i * 24 for i in range(10)]

    for day in weatherData["forecast"]["forecastday"]:
        for hour in day["hour"]:
            
            hour_time = hour["time"]
            hours.append(hour_time)
            hour_labels.append(hour_time[-5:]) 
            tempC.append(hour["temp_c"])
            feelslikeC.append(hour["feelslike_c"])
            windKph.append(hour["wind_kph"])
            gustKph.append(hour["gust_kph"])
            precipMm.append(hour["precip_mm"])
            snowCm.append(hour.get("snow_cm", 0))
            humidity.append(hour["humidity"])
            cloud.append(hour["cloud"])
            windchillC.append(hour["windchill_c"])
            heatindexC.append(hour["heatindex_c"])
            dewpointC.append(hour["dewpoint_c"])
            rainChance.append(hour["chance_of_rain"])
            snowChance.append(hour["chance_of_snow"])
            visibilityKm.append(hour["vis_km"])
            uvIndex.append(hour["uv"])

    def filter_xticks(hours):
        xticks = []
        xlabels = []
        for i, h in enumerate(hours):
            hour = int(h[-5:-3])
            if hour in [0, 4, 8, 12, 16, 20]:
                xticks.append(i)
                xlabels.append(h[-5:])
        return xticks, xlabels

    def plotSubplot(data, titles, i):
        plt.subplot(6, 1, i)
        for d, t in zip(data, titles):
            plt.plot(hours, d, label=t)
        xticks, xlabels = filter_xticks(hours)
        plt.xticks(xticks, xlabels, rotation=45)
        for boundary in dayends:
            plt.axvline(x=boundary, color='gray', linestyle='--', alpha=0.5)
            di = dayends.index(boundary)
            dl = weatherData["forecast"]["forecastday"][di]["date"]
            plt.text(boundary + 1, plt.ylim()[1], dl, rotation=0, ha='left', va='bottom', fontsize=8, color='gray')

        plt.legend()
        plt.title(' / '.join(titles))

    plt.figure(figsize=(16, 24))
    plotSubplot([tempC, feelslikeC], ["Temperature (°C)", "Feels Like (°C)"], 1)
    plotSubplot([windKph, gustKph], ["Wind Speed (kph)", "Gust Speed (kph)"], 2)
    plotSubplot([precipMm, snowCm], ["Precipitation (mm)", "Snow (cm)"], 3)
    plotSubplot([humidity, cloud], ["Humidity (%)", "Cloud Cover (%)"], 4)
    plotSubplot([windchillC, heatindexC, dewpointC], ["Wind Chill (°C)", "Heat Index (°C)", "Dew Point (°C)"], 5)
    plotSubplot([rainChance, snowChance], ["Chance of Rain (%)", "Chance of Snow (%)"], 6)
    plt.tight_layout()
    plt.show()

    fig, ax1 = plt.subplots(figsize=(12, 5))
    ax2 = ax1.twinx()
    ax1.plot(hours, visibilityKm, 'g-', label="Visibility (km)")
    ax2.plot(hours, uvIndex, 'b-', label="UV Index")
    xticks, xlabels = filter_xticks(hours)
    ax1.set_xticks(xticks)
    ax1.set_xticklabels(xlabels, rotation=45)
    for boundary in dayends:
        ax1.axvline(x=boundary, color='gray', linestyle='--', alpha=0.5)
        di = dayends.index(boundary)
        dl = weatherData["forecast"]["forecastday"][di]["date"]
        ax1.text(boundary + 1, ax1.get_ylim()[1], dl, rotation=0, ha='left', va='bottom', fontsize=8, color='gray')

    ax1.set_xlabel("Time")
    ax1.set_ylabel("Visibility (km)", color='g')
    ax2.set_ylabel("UV Index", color='b')
    plt.title("Visibility and UV Index")
    plt.tight_layout()
    plt.show()


Level 1 and 3

In [203]:
def runWeatherMind(user_input):
    result = weatherMind.invoke({"input": user_input})
    print(f"\nTask: {result.get('task')}")
    print(f"Output: {result.get('output')}")
    
    if 'weatherData' in result:
        plotWeatherGraphs(result['weatherData'])
    
    return result

# runWeatherMind("how to not feel suicidal after reading documentation")
# runWeatherMind("Can you calculate 94 * 2 + 3? and also 24 /5 and also 2**3")
# runWeatherMind("Tell me about quantum computing")
# runWeatherMind("What is the weather like in London?")
# runWeatherMind("Wzhat is the weather like in New Amsterdam?")
runWeatherMind("What is the current fashion like in Tokyo?")


Task: None
Output: No overview found.


{'output': 'No overview found.',
 'images': ['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQUVLI5rn3cyxOGVyilM_U_aQ7kB9PmEyqNsahXPlApFEj7swXqIh1PQQP3bg&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSpSAboNGEbRkEyf27o46pThzOJCbCimFtuXzxMwdlIOpq7AvIAG0aElunBww&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSSBGq92D_1Svbw99n2_DY7zVBFPaCtOhyoB4O_CK6f1DngN-d_jR-9E_1dFg&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSf4YWORgGwgbSa59_yUV_RWfQ2MRT-nGp4YC_3wq9t1y5kMXQ9icGGmAuJe10&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRDC2qlyQdUW7J2pbHdr_OjXZMP6N5k2oJKK9lVyf60Ia-Dn_uBfp5m8ee_mQ&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT_Grur6E7_BPOOhZtQHK0oqjBbxu3iQqhs59JBrlA2zGafu-U4yFBkwHdn7Mg&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGILiKNSBE7q-tV0mYn20A8QhZLjzhjwdY8pv-DSmrKJrP6uhFsy6DVynwcDE&s',
  'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRb6e8uskWRG3kWb5AGFszic-_GYLJT79NDRzhKifbd2

In [204]:
runWeatherMind("explain the historical context behind the semi riddle last msg?")


Task: llm
Output: llm
The term "semi riddle" is not a standard or widely recognized phrase in historical or cultural studies, which makes it a bit challenging to provide a precise historical context. However, I can break down the possible components and offer some insights.

### Possible Interpretations:
1. **Semi as in Semiramis**:
   - **Semiramis** is a legendary figure from ancient Assyrian history. She is often associated with myths and legends, including the founding of various cities and monuments. If "semi" refers to Semiramis, the "riddle" could be related to the myths and legends surrounding her life and deeds.

2. **Semi as in Semiotics**:
   - **Semiotics** is the study of signs and symbols and their use or interpretation. If "semi" refers to semiotics, the "riddle" could be related to the interpretation of symbols and signs in historical contexts, such as ancient texts, artifacts, or cultural practices.

3. **Semi as in Semi-Precious**:
   - If "semi" refers to semi-preci

{'output': 'llm\nThe term "semi riddle" is not a standard or widely recognized phrase in historical or cultural studies, which makes it a bit challenging to provide a precise historical context. However, I can break down the possible components and offer some insights.\n\n### Possible Interpretations:\n1. **Semi as in Semiramis**:\n   - **Semiramis** is a legendary figure from ancient Assyrian history. She is often associated with myths and legends, including the founding of various cities and monuments. If "semi" refers to Semiramis, the "riddle" could be related to the myths and legends surrounding her life and deeds.\n\n2. **Semi as in Semiotics**:\n   - **Semiotics** is the study of signs and symbols and their use or interpretation. If "semi" refers to semiotics, the "riddle" could be related to the interpretation of symbols and signs in historical contexts, such as ancient texts, artifacts, or cultural practices.\n\n3. **Semi as in Semi-Precious**:\n   - If "semi" refers to semi-p